In [ ]:
import torch, os, datetime, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms 
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random
import pandas as pd
import time
from sklearn.metrics import precision_score, recall_score, f1_score

import seaborn as sns
import optuna
from optuna.trial import TrialState
import inspect
import importlib

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Carregando e seperando dataset

In [ ]:
batch_size = 32

# Transformações para normalizar o dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Carregando o dataset MNIST
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader para o dataset de treinamento e teste
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

classes = list(map(str, range(0, 10)))

##### Visualizando dados

In [ ]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))
        break

In [ ]:
show_batch(trainloader)

In [ ]:
show_batch(testloader)

## Criando uma instância da classe nn.Module para criar redes

#### Explicando cada parâmetro da camada convolucional:

- **in_channels**: Quantos canais de cores os inputs possuem. No caso de imagens em preto e branco, como estamos trabalhando com MNIST, há apenas um canal de cor.

- **out_channels**: Número de filtros (kernels) que serão aplicados à imagem durante a convolução. Cada filtro é responsável por extrair características latentes da imagem.

- **kernel_size**: Dimensão do filtro utilizado na convolução. O valor comum de 3 é amplamente usado, pois é suficiente para capturar detalhes locais, ao mesmo tempo em que percebe padrões maiores na imagem.

- **stride**: Indica de quanto em quantos pixels o filtro será aplicado na imagem.

- **padding**: Adiciona pixels em volta da imagem de entrada durante a convolução para garantir que o tamanho da saída após a convolução permaneça o mesmo que o da entrada.
  - *Observação*: Ao adicionar padding, é preciso ter cuidado com os valores para garantir que a resolução e o tamanho da imagem não sejam afetados de forma indesejada.

- **dropout_prob**: Probabilidade de que um neurônio seja desligado durante o treinamento da rede. Isso é uma técnica de regularização que ajuda a prevenir o overfitting, forçando a rede a aprender representações mais robustas e generalizáveis.

#### Fórmulas das Dimensões de Saída

A fórmula para calcular a dimensão de saída \( W_out \) da convolução é:

$$
W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} + 2 \times \text{padding} - \text{kernel\_size}}{\text{stride}} \right\rfloor + 1
$$

E a fórmula para calcular a dimensão de saída \( H_out \) da convolução é:

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2 \times \text{padding} - \text{kernel\_size}}{\text{stride}} \right\rfloor + 1
$$

Resumindo as trasnformações da BaseCNN:

- Entrada Original: 28x28 pixels, 1 canal.
- Após Convolução: 28x28 pixels, 32 canais.
- Após Pooling: 14x14 pixels, 32 canais.
- Entrada para self.fc1: 32 × 14 × 14

A BaseCNN é uma implementação básica de uma rede neural convolucional (CNN). Ela possui uma camada de convolução, uma camada de max pooling, uma camada de dropout e uma camada fully connected (linear). Esta arquitetura foi desenvolvida para experimentos iniciais.

In [ ]:
class BaseCNN(nn.Module):
    def __init__(self, conv_kernel_size=3, conv_stride=1, conv_padding=1, pool_kernel_size=2, pool_stride=2, dropout_prob=0.5):
        super(BaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=conv_kernel_size, stride=conv_stride, padding=conv_padding)
        self.pool = nn.MaxPool2d(kernel_size=pool_kernel_size, stride=pool_stride)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc1 = nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(-1, 32 * 14 * 14)
        x = self.fc1(x)
        return x

Já a classe CNN é uma rede neural convolucional mais complexa que leva em consideração os hiperparâmetros sugeridos pelo Optuna para otimização automática. Ela possui várias camadas convolucionais, uma camada de pooling, camadas fully connected (lineares) e camadas de dropout. O método calc_conv_output_size é usado para calcular o tamanho da saída após as operações de convolução e pooling.

A rede começa com a entrada passando por camadas convolucionais e max pooling, onde os dados são filtrados e as dimensões são reduzidas. Depois, um dropout 2D é aplicado após a terceira camada convolucional para regularizar o treinamento. Os dados são então remodelados para as camadas fully connected, onde são aplicadas ativações ReLU e dropout antes de passar para a camada final que gera as previsões. Antes de retornar as previsões, uma função Log Softmax é aplicada para normalizar as saídas e gerar uma distribuição de probabilidade sobre as classes.

In [ ]:
class CNN(nn.Module):
    def _calc_conv_output_size(self, in_size, kernel_size, conv_stride, pool_size, pool_stride, conv_padding):
        # Apply convolution
        out_size = (in_size + 2 * conv_padding - kernel_size) // conv_stride + 1
        # Apply pooling
        out_size = (out_size - pool_size) // pool_stride + 1
        return out_size

    def __init__(self, num_conv_layers, num_filters, num_neurons, drop_conv2, drop_prob_fc1,
                 conv_kernel_size=3, pool_kernel_size=2, conv_stride=1, pool_stride=2, conv_padding=0):
        super(CNN, self).__init__()

        in_size = 28

        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters[0],
                                              kernel_size=(conv_kernel_size, conv_kernel_size),
                                              stride=conv_stride, padding=conv_padding)])
        out_size = self._calc_conv_output_size(in_size, conv_kernel_size, conv_stride,
                                               pool_kernel_size, pool_stride, conv_padding)

        for i in range(1, num_conv_layers):
            self.convs.append(nn.Conv2d(in_channels=num_filters[i - 1], out_channels=num_filters[i],
                                        kernel_size=(conv_kernel_size, conv_kernel_size),
                                        stride=conv_stride, padding=conv_padding))
            out_size = self._calc_conv_output_size(out_size, conv_kernel_size, conv_stride,
                                                   pool_kernel_size, pool_stride, conv_padding)

        if out_size <= 0:
            raise ValueError("Output size is too small after convolution and pooling operations. " +
                             "Adjust the kernel sizes and strides.")

        self.conv2_drop = nn.Dropout2d(p=drop_conv2)
        self.out_feature = num_filters[num_conv_layers - 1] * out_size * out_size
        self.pool = nn.MaxPool2d(kernel_size=pool_kernel_size, stride=pool_stride)
        self.fc1 = nn.Linear(self.out_feature, num_neurons)
        self.fc2 = nn.Linear(num_neurons, 10)
        self.fc1_drop_prob = drop_prob_fc1

    def forward(self, x):
        for i, conv in enumerate(self.convs):
            x = conv(x)
            if i == 2:
                x = self.conv2_drop(x)
            x = F.relu(self.pool(x))

        x = x.view(-1, self.out_feature)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.fc1_drop_prob, training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

# Definindo funções úteis

#### Função de treinamento do modelo

In [ ]:
def train_model(model, criterion, optimizer, epochs):
    model.train().to(device)

    for epoch in range(epochs):
        running_loss = 0.0

        for inputs, labels in trainloader:
            (inputs, labels) = (inputs.to(device), labels.to(device))
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')
    
    print('Finished Training!')

#### Função de teste do modelo

In [ ]:
def test_model(model, criterion):
    model.eval().to(device)
    correct = 0
    total = 0
    test_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = correct / total
    loss = test_loss / len(testloader)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    return accuracy, loss, precision, recall, f1

#### Função de acurácia de cada classe

In [ ]:
def accuracy_classes(model_trained):
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    all_accuracy = []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_trained(images)
            _, predictions = torch.max(outputs, 1)
            
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        all_accuracy.append(accuracy)
    
    return all_accuracy

#### Função para prever classe de uma imagem

In [ ]:
def predict_image(image_path, model):
    image = Image.open(image_path).convert('L')  
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor()
    ])

    image = transform(image).unsqueeze(0) 

    output = model(image).to(device)

    _, predicted_class = torch.max(output, 1)
    print("Classe prevista:", predicted_class.item())


#### Função de treinamento por epoca

In [ ]:
def train_model_once(model, criterion, optimizer):
    model.train().to(device)

    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
 
    return running_loss / len(trainloader)

# Fine-Tuning de Hiperparâmetros

Em seguida, realizamos 10 experimentos, cada um com parâmetros diferentes escolhidos usando a biblioteca Optuna, para otimização dos hiperparâmetros. Para cada experimento, iremos salvar seus resultados (acurácia total, perda de teste, precisão, recall e f1 score), e quais os parâmetros utlizados (número de camadas convolucionais, número de filtros, número de neurônios, dropout para a segunda camada convolucional, probabilidade de dropout para a primeira camada totalmente conectada, tamanho do kernel convolucional, stride convolucional, padding convolucional, tamanho do kernel de pooling, stride de pooling).
<br>No final de cada experimento, são salvos os três modelos com a maior acurácia total, os quais são exibidos após a conclusão de todos os 10 testes.

In [ ]:
def objective(trial):
    trial.set_user_attr("network", type(model_class).__name__)
    trial.set_user_attr("total_epochs", num_epochs)

    # Define range of values to be tested for the hyperparameters
    trial_params = {
        "num_conv_layers": trial.suggest_int("num_conv_layers", 1, 3),
        "num_filters": [int(trial.suggest_discrete_uniform("num_filter_" + str(i), 16, 128, 16))
                        for i in range(trial.suggest_int("num_conv_layers", 1, 3))],
        "num_neurons": trial.suggest_int("num_neurons", 10, 400, 10),       # Number of neurons of fully connected layer 1
        "drop_conv2": trial.suggest_float("drop_conv2", 0.2, 0.5),          # Dropout for convolutional layer 2
        "drop_prob_fc1": trial.suggest_float("drop_prob_fc1", 0.2, 0.5),    # Dropout probability for fully connected layer 1
        "conv_kernel_size": trial.suggest_int("conv_kernel_size", 3, 7, step=2),
        "conv_stride": trial.suggest_int("conv_stride", 1, 2),
        "conv_padding": trial.suggest_int("conv_padding", 1, 3),
        "pool_kernel_size": trial.suggest_int("pool_kernel_size", 2, 3),
        "pool_stride": trial.suggest_int("pool_stride", 2, 3),
        "dropout_prob": trial.suggest_float("drop_prob_fc1", 0.2, 0.5)
    }

    # Manually add these parameters to the trial's attributes
    trial.set_user_attr("conv_kernel_size", trial_params["conv_kernel_size"])
    trial.set_user_attr("conv_stride", trial_params["conv_stride"])
    trial.set_user_attr("conv_padding", trial_params["conv_padding"])
    trial.set_user_attr("pool_kernel_size", trial_params["pool_kernel_size"])
    trial.set_user_attr("pool_stride", trial_params["pool_stride"])

  
    # Generate the model
    model_params = inspect.signature(model_class.__init__).parameters
    model_init_args = {}
    
    for param_name, param in model_params.items():
        if param_name == 'self':
            continue
        else:
            model_init_args[param_name] = trial_params[param_name]

    try:
        model = model_class(**model_init_args).to(device)

        # Generate the optimizer
        optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
        lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

        # Train the model
        for epoch in range(num_epochs):
            train_loss = train_model_once(model, criterion, optimizer)
            accuracy, test_loss, precision, recall, f1 = test_model(model, criterion)

            print(f"Epoch {epoch + 1}, " +
                f"Train Loss: {train_loss:.4f}, " + 
                f"Test Loss: {test_loss:.4f}")

            # Pruning (stops trial early if not promising)
            trial.report(accuracy, epoch)
            # Handle pruning based on the intermediate value.
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        current_datetime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        path_model_trained = os.path.join('modelos_treinados', f'optune{current_datetime}.pth')
        torch.save(model.state_dict(), path_model_trained)

        trial.set_user_attr("accuracy", accuracy)
        trial.set_user_attr("test_loss", test_loss)
        trial.set_user_attr("precision", precision)
        trial.set_user_attr("recall", recall)
        trial.set_user_attr("path_model_trained", path_model_trained)

        
        return accuracy
    except:
        return None

# Criando nossa rede

In [ ]:
model_class = BaseCNN #or other class
num_epochs = 10
num_trials = 10
criterion = nn.CrossEntropyLoss().to(device)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

### Savando os resultados obtidos no csv e ordenando por acurácia

In [ ]:
df = study.trials_dataframe()
df = df[df["state"] == "COMPLETE"].drop(["datetime_start", "datetime_complete", "duration", "state"], axis=1)
df = df.sort_values("value")  # Sort based on accuracy

csv_file = "optuna_results.csv"
df_existing = pd.read_csv(csv_file) if os.path.isfile(csv_file) else None

df_existing = pd.concat([df_existing, df], ignore_index=True) if df_existing is not None else df
df_existing.to_csv(csv_file, index=False)

In [ ]:
df_existing

# Estatísticas

In [ ]:
print("\n-- Study Statistics --")
print(f"  Number of finished trials: {len(study.trials)}")
print(f"  Number of pruned trials:   {len(pruned_trials)}")
print(f"  Number of complete trials: {len(complete_trials)}")

best_trial = study.best_trial
print("\n-- Best Trial --")
print(f"  Accuracy:  {best_trial.value}")
print(f"  Test Loss: {best_trial.user_attrs['test_loss']}")
print(f"  Precision: {best_trial.user_attrs['precision']}")
print(f"  Recall:    {best_trial.user_attrs['recall']}")
print(f"  F1 Score:  {best_trial.user_attrs['f1_score']}")
print("  Parameters: ")
for key, val in best_trial.params.items():
    print(f"    {key}: {(16 - len(key)) * ' '}{val}")
print(f"  conv_kernel_size: {best_trial.user_attrs['conv_kernel_size']}")
print(f"  conv_stride:      {best_trial.user_attrs['conv_stride']}")
print(f"  conv_padding:     {best_trial.user_attrs['conv_padding']}")
print(f"  pool_kernel_size: {best_trial.user_attrs['pool_kernel_size']}")
print(f"  pool_stride:      {best_trial.user_attrs['pool_stride']}")

print(f"\n-- Overall Results (Ordered by Accuracy) --")
print(df)

most_important_parameters = optuna.importance.get_param_importances(study, target=None)

print("\n-- Most Important Hyperparameters --")
for key, val in most_important_parameters.items():
    print(f"  {key}: {(15 - len(key)) * ' '}{(100 * val):.2f}%")

# One Model

#### Instânciando o modelo base, definindo parametros, loss function e otimizador

In [ ]:
model = BaseCNN().to(device)

learning_rate = 0.001
epochs = 10

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

#### Treinamento o modelo

In [ ]:
train_model(model, criterion, optimizer, epochs)

#### Testando o modelo

In [ ]:
accuracy, loss, precision, recall, f1 = test_model(model, criterion)
print(f'Accuracy: {100 * accuracy}%, Test Loss: {loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

#### Salvando o modelo treinado

In [ ]:
if not os.path.exists('modelos_treinados'):
    os.makedirs('modelos_treinados')

current_datetime = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
path_model_trained = os.path.join('modelos_treinados', f'model_{current_datetime}.pth')

torch.save(model.state_dict(), path_model_trained)

## Carregando modelo treinado

In [ ]:
path_model_trained = 'modelos_treinados\optune20240522_025008.pth'
path_csv = "src\optuna_results.csv"
df = pd.read_csv(path_csv)

row = df[df['path_model_trained'] == path_model_trained]
className = row['network'].iloc[0]
ModelClass = getattr(importlib.import_module('__main__'), className)

model_trained = ModelClass().to(device)
model_trained.load_state_dict(torch.load(path_model_trained))
model_trained.eval()

#### Analisando acurácia de cada classe

In [ ]:
all_accuracy = accuracy_classes(model_trained)
for i, accuracy in enumerate(all_accuracy):
    print(f'Accuracy for class: {i} is {accuracy:.2f} %')

# Análise dos Resultados

In [ ]:
df = pd.read_csv('optuna_results.csv')
df.sort_values(by=['number'], inplace=True)
df.columns

In [ ]:
df

In [ ]:
x = np.arange(len(df['number']))
width = 0.2  # Largura das barras

fig, ax = plt.subplots()
bars4 = ax.bar(x - width, df['user_attrs_accuracy'], width - 0.01, label='Acurácia', color=(0.9, 0.41, 0.38))
bars2 = ax.bar(x, df['user_attrs_precision'], width - 0.01, label='Precisão', color=(1.0, 0.8, 0.6, 0.8))
bars1 = ax.bar(x + width, df['user_attrs_recall'], width - 0.01, label='Recall', color=(0.6, 0.8, 1.0, 0.8))
bars3 = ax.bar(x + 2*width, df['user_attrs_f1_score'], width - 0.01, label='F1-Score', color=(0.6, 1.0, 0.8, 0.8))

ax.set_xlabel('Modelos')
ax.set_ylabel('Métricas')
ax.set_title('Métricas de Avaliação de Modelos')
ax.set_xticks(x)
ax.set_xticklabels(df['number'])
ax.legend()

In [ ]:
fig, ax = plt.subplots()
bars4 = ax.bar(x - width, df['user_attrs_accuracy'], width - 0.01, label='Acurácia', color=(0.9, 0.41, 0.38))
bars2 = ax.bar(x, df['user_attrs_precision'], width - 0.01, label='Precisão', color=(1.0, 0.8, 0.6, 0.8))
bars1 = ax.bar(x + width, df['user_attrs_recall'], width - 0.01, label='Recall', color=(0.6, 0.8, 1.0, 0.8))
bars3 = ax.bar(x + 2*width, df['user_attrs_f1_score'], width - 0.01, label='F1-Score', color=(0.6, 1.0, 0.8, 0.8))

ax.set_xlabel('Modelos')
ax.set_ylabel('Métricas')
ax.set_title('Métricas de Avaliação de Modelos')
ax.set_xticks(x)
ax.set_xticklabels(df['number'])
ax.set_ylim(0.9, 1.0)
ax.legend()

In [ ]:
acc_data = pd.DataFrame(all_accuracy, columns=['Acurácia'])

plt.figure(figsize=(10, 6))
plt.bar(acc_data.index, acc_data['Acurácia'], color='skyblue')

# Adicionando títulos e rótulos
plt.title('Acurácia por Classe')
plt.xlabel('Classe')
plt.ylabel('Acurácia (%)')
plt.xticks(acc_data.index)
plt.ylim(80, 100)

# Mostrando o gráfico
plt.show()

In [ ]:
# Agrupar por otimizador e calcular médias
optimizer_group = df.groupby('params_optimizer').mean()

r1 = np.arange(len(optimizer_group.index))
r2 = [x + width for x in r1]
r3 = [x + width for x in r2]


# Gráfico de barras
plt.figure(figsize=(14, 8))

# Precisão
plt.bar(r1, optimizer_group['user_attrs_precision'], width - 0.01, label='Precisão', alpha=0.6)

# Recall
plt.bar(r2, optimizer_group['user_attrs_recall'], width - 0.01, label='Recall', alpha=0.6)

# F1-score
plt.bar(r3, optimizer_group['user_attrs_f1_score'], width - 0.01, label='F1-Score', alpha=0.6)

plt.xlabel('Otimizador')
plt.ylabel('Média dos Valores')
plt.title('Média de Precisão, Recall e F1-Score para Cada Otimizador')
plt.xticks([r + width for r in range(len(optimizer_group.index))], optimizer_group.index)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df['number'], df['user_attrs_test_loss'], marker='o', linestyle='-', color='red')
plt.xlabel('Número do Experimento')
plt.ylabel('Perda de Teste')
plt.title('Perda de Teste vs. Número do Experimento')
plt.legend(['Teste'])
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))

sns.scatterplot(data=df, x='params_num_conv_layers', y='user_attrs_accuracy', hue='params_optimizer', style='params_optimizer', markers=True,
                palette='viridis', s=100)

plt.title('Comparação de Acurácia dos Modelos por Quantidade de Camadas Convolucionais')
plt.xlabel('Número de Camadas Convolucionais')
plt.ylabel('Precisão')
plt.xticks(df['params_num_conv_layers'].unique())
plt.grid(True)
plt.legend(title='Tipo de otimizador', bbox_to_anchor=(1.05, 1), loc='upper left')  # Movendo a legenda para fora do gráfico

plt.show()

In [ ]:
data = pd.DataFrame(df.groupby('params_num_conv_layers')['user_attrs_accuracy'].mean().reset_index())

In [ ]:
plt.figure(figsize=(16, 8))

sns.scatterplot(data=data, x='params_num_conv_layers', y='user_attrs_accuracy', markers=True,
                 s=100)

plt.title('Média da Acurácia por Quantidade de Camadas Convolucionais')
plt.xlabel('Número de Camadas Convolucionais')
plt.ylabel('Precisão')
plt.xticks(df['params_num_conv_layers'].unique())
plt.grid(True)

plt.show()

In [ ]:

plt.figure(figsize=(10, 6))
sns.countplot(x='params_num_conv_layers', hue='params_optimizer', data=df)
plt.title('Distribuição das Camadas Convolucionais por Otimizador')
plt.xlabel('Número de Camadas Convolucionais')
plt.ylabel('Contagem')
plt.legend(title='Otimizador')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))

sns.scatterplot(data=df, x='params_drop_conv2', y='user_attrs_accuracy', hue='params_num_conv_layers', palette='viridis'
                , s=100)

plt.title('Acurácia por Taxa de Dropout')
plt.xlabel('Taxa de dropout')
plt.ylabel('Acurácia')
plt.xlim(0.0, 1)
plt.grid(True)
plt.legend(title='Número de Camadas Convolucionais', loc='upper right')

plt.show()